In [ ]:
# 참조 : https://code.activestate.com/recipes/473810-send-an-html-email-with-embedded-image-and-plain-t/

In [1]:
###############
# 3.이메일 발송 : gamil
#####
import pandas as pd
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders
from email.header import Header
from datetime import datetime, timedelta
from email.mime.image import MIMEImage
from jinja2 import Template

gmail_user= '**********'
gmail_pwd = '**********'


In [2]:
def send_gmail(to, subject, html, graph=None, attach=None):
    msg=MIMEMultipart('related')
    msg['From']=gmail_user
    msg['To'] = to
    msg['Subject'] = Header(s=subject, charset="utf-8")
    msg.attach(MIMEText(html, 'html', _charset="utf-8"))
    
    # 이미지 내용 추가(1장만 가능한 format)
    if graph:
        fp = open(graph, 'rb')
        msgImage = MIMEImage(fp.read())
        fp.close()
        msgImage.add_header('Content-ID', '<image1>')
        msg.attach(msgImage)

    #첨부파일
    if attach:
        part=MIMEBase('application','octet-stream')
        part.set_payload(open(attach, 'rb').read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition','attachment; filename="%s"' % os.path.basename(attach))
        msg.attach(part)
     
    s = smtplib.SMTP("smtp.gmail.com", 587)
    s.ehlo()
    s.starttls()
    s.ehlo()
    s.login(gmail_user, gmail_pwd)
    s.sendmail(gmail_user, to, msg.as_string())
    s.close()

In [5]:
# 이메일 양식(내용) frame 구성하기
message_tmpl="""
<html>
    <!DOCTYPE html>
    <html>
    <head lang ="ko">
      <meta charset ="UTF-8">
      <title>{{title1}}</title>
    </head>
    <BODY style="FONT-SIZE: 11pt; FONT-FAMILY: 돋움">
    안녕하세요,<br>
    <br>
    금일({{d0}} ) 쿠팡에서 멸균우유 가격 결과 공유드립니다.<br>
    <img src="{{ filename='./작업스케쥴러2_트리거.JPG' }}">

    {{title2}}<br>
    {{title2_text2}}<br>
    {{ df2_html }}<br/>
    <img src="{{ filename1 }}">

    {{title3}}
    <br><img src="cid:image1"><br>
    {{title3_text1}}<br>
    <br/>
    감사합니다.
    </BODY>
</html>
"""

title2 = "1. 멸균우유 가격 추이(최근 1주일)"
title3 = "2. 멸균우유 가격 추이(그래프)"

render_data = {
    'd0' : (datetime.today()).strftime('%Y-%m-%d, %H:%M'),
    'title2' : title2,
    'filename1' : 

#     'title2_text1' : " - graph",
#     'df1_html' : '(추후 업데이트)',
#     'title2_text2' : " - table",
#     'df2_html' : df2.to_html(),
    'title3' : title3,
    'title3_text1' : "* 이메일 수신?"
}


t= Template(message_tmpl)
html_report = t.render(render_data)

TemplateSyntaxError: expected token 'end of print statement', got '='

In [75]:
to_users = ['khpark@maeil.com']

# 받는사람 : mulit 발송
# to_users = ['khpark@maeil.com', 'mj@maeil.com', 'kangjongw@maeil.com', 'minjungkeem@maeil.com']

# 메일제목
now = datetime.today()    # 현재
date = now.strftime('%Y%m%d')

subject = "[test_v3] "+date[2:] + "_쿠팡 가격 리포트(발송전용)"

# 본문
html = html_report

attach = './200925_coupang_graph.png'
graph = './200925_coupang_graph.png'

In [76]:
for to_users in to_users :
    send_gmail(to_users, subject, html, graph, attach)